In [93]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key = os.environ['OPENAI_API_KEY']
openai.base_url = os.environ['OPENAI_API_BASE']

In [94]:
from langchain.chat_models import ChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm = ChatOpenAI(temperature=0.9, model='gpt-4o')


In [95]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [96]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [97]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [98]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [99]:
chain = LLMChain(llm=llm, prompt=prompt)
product = "Queen Size Sheet Set"
result = chain.run(product)

In [100]:
from IPython.display import display, Markdown
Markdown(result)

When naming a company that specializes in Queen Size Sheet Sets, you'll want to consider names that convey comfort, quality, and luxury. Here are a few suggestions:

1. **Queen's Comfort**
2. **Regal Rest Linens**
3. **Harmony Sheets**
4. **Royal Dream Linens**
5. **Majestic Sheets**
6. **Sovereign Sleep**
7. **Comfort Crown Linens**
8. **Empress Bedding**
9. **Noble Night Sheets**
10. **LuxQueen Linens**

These names emphasize the quality and luxury associated with queen size bedding while being catchy and memorable.

In [101]:
from langchain.chains import SimpleSequentialChain

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [102]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [103]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [104]:
result = overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Choosing a name for a company that makes Queen Size Sheet Sets involves emphasizing qualities like comfort, luxury, quality, and elegance. Here are some name ideas that might resonate well:

1. **Queen's Comfort**
2. **LuxuLinen**
3. **RoyalRest**
4. **Harmony Sheets**
5. **DreamWeave**
6. **Serene Slumber**
7. **Elegance Linens**
8. **Majesty Sheets**
9. **QueenQuilt**
10. **SoftNest**

When selecting a name, consider the values and image you want your brand to convey, and ensure it's unique and memorable. Additionally, check the availability of domain names and trademarks to ensure exclusivity.
Our company delivers luxurious, high-quality Queen Size Sheet Sets that epitomize comfort, elegance, and style for a serene sleep experience.

> Finished chain.


In [105]:
Markdown(result)

Our company delivers luxurious, high-quality Queen Size Sheet Sets that epitomize comfort, elegance, and style for a serene sleep experience.

In [106]:
from langchain.chains import SequentialChain

In [107]:

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [108]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [109]:
review = df.Review[1]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'English_Review': 'I loved the waterproof bag, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on once it was in the pouch.',
 'summary': 'The waterproof bag was appreciated for its effectiveness, though concerns were raised about the durability of the hard plastic opening and its impact on phone usability.',
 'followup_message': "Thank you for your feedback on the waterproof bag. We’re glad to hear that you found it effective in keeping your belongings safe and dry. Your concerns about the durability of the hard plastic opening are important to us, and we'll take them into consideration for future product improvements. Additionally, we understand the need for better phone usability and are exploring options to enhance this asp

In [110]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [111]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }

    
]

In [112]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [113]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [114]:
destinations

['physics: Good for answering questions about physics',
 'math: Good for answering math questions',
 'History: Good for answering history questions',
 'computer science: Good for answering computer science questions']

In [115]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [116]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [117]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
/var/folders/q6/2kbg8jyx09ngj0srzvvbd72c0000gn/T/ipykernel_35970/2160379643.py:1: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [118]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [119]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [120]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by an idealized object, known as a black body, which absorbs all incident radiation, regardless of frequency or angle of incidence. By definition, a perfect black body does not reflect or transmit any light; it is a perfect absorber. \n\nWhen a black body is at a uniform temperature, it emits radiation in a characteristic spectrum, which only depends on its temperature. This spectrum is described by Planck's law and peaks at a wavelength inversely proportional to the temperature, a relationship explained by Wien's displacement law. The total energy emitted across all wavelengths scales with the fourth power of the temperature, as described by the Stefan-Boltzmann law. \n\nBlack body radiation is a key concept in the study of thermodynamics and quantum mechanics, and it played a crucial role in the development of quantum theory."

In [121]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.


'To solve the question "What is 2 + 2?", we can break it down into its basic arithmetic components. We start with the two numbers: 2 and 2.\n\n1. The operation we need to perform is addition.\n2. Addition is a mathematical operation that combines two numbers into a single sum.\n3. When we add 2 and 2 together, we find their sum by counting up from 2.\n\nLet\'s calculate:\n- Start with the number 2.\n- Add 2 to it: 2 + 2 = 4.\n\nTherefore, the answer to the question "What is 2 + 2?" is 4.'

In [122]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because it serves as the central repository of genetic information necessary for the development, functioning, and reproduction of the organism. Here are a few reasons why DNA is present in nearly every cell:\n\n1. **Genetic Blueprint**: DNA contains the genes that encode proteins, which are critical for virtually every cellular function. Each cell needs access to this genetic blueprint to perform its specialized roles effectively.\n\n2. **Cell Function and Specialization**: Although all cells contain the same DNA, different genes are expressed in different cell types, allowing for cell specialization. This means muscle cells, nerve cells, and blood cells can all perform their unique functions despite having the same genetic information.\n\n3. **Cell Division**: DNA replication is essential for cell division, which is how organisms grow and repair damaged tissues. Each new cell needs a complete copy of the DNA to function properly.\n\n4. **Regulatio